## **Column Transformer**

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [2]:
df = pd.read_csv(r"C:\Users\singh\OneDrive\Desktop\Python\Data\covid_toy.csv")
df.head(10)

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
5,84,Female,NaN,Mild,Bangalore,Yes
6,14,Male,101.0,Strong,Bangalore,No
7,20,Female,NaN,Strong,Mumbai,Yes
8,19,Female,100.0,Strong,Bangalore,No
9,64,Female,101.0,Mild,Delhi,No


In [3]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [4]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns =
                                                            ['has_covid']), df['has_covid'], test_size = 0.2, random_state = 59)

In [5]:
x_train

,age,gender,fever,cough,city
47,18,Female,104.0,Mild,Bangalore
34,74,Male,102.0,Mild,Mumbai
28,16,Male,104.0,Mild,Kolkata
19,42,Female,NaN,Strong,Bangalore
81,65,Male,99.0,Mild,Delhi
...,...,...,...,...,...
79,48,Female,103.0,Mild,Kolkata
23,80,Female,98.0,Mild,Delhi
77,8,Female,101.0,Mild,Kolkata
44,20,Male,102.0,Strong,Delhi


### Manully Type Output

In [6]:
## adding simple imputer to fever column
si = SimpleImputer(strategy= "mean")
x_train_fever = si.fit_transform(x_train[['fever']])

# also the test data
x_test_fever = si.fit_transform(x_test[['fever']])

x_train_fever.shape

(80, 1)

In [7]:
## Ordinal Encoding ---> Cough

oe = OrdinalEncoder(categories= [['Mild', 'Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

# also the test data
x_test_cough = oe.fit_transform(x_test[['cough']])

x_train_cough.shape

(80, 1)

In [8]:
## OneHotEncoding ---> Gender, City

ohe =OneHotEncoder(drop = 'first', sparse_output= False)
x_train_gender_city = ohe.fit_transform(x_train[['gender', 'city']])

# also the test data
x_test_gender_city = ohe.fit_transform(x_test[['gender', 'city']])

x_train_gender_city.shape

(80, 4)

In [9]:
# Extracting Age

x_train_age = x_train.drop(columns= ['gender', 'fever', 'cough', 'city']).values

# also the test data
x_test_age = x_test.drop(columns = ['gender', 'fever', 'cough', 'city']).values

In [10]:
x_train_age.shape

(80, 1)

In [11]:
x_train_transformed = np.concatenate((x_train_age, x_train_fever,
                                      x_train_gender_city,
                                      x_train_cough), axis = 1)

In [12]:
x_train_transformed.shape

(80, 7)

### by the help of Column Transformer

In [32]:
from sklearn.compose import ColumnTransformer

In [14]:
transformer = ColumnTransformer(transformers=[('tnf1', SimpleImputer(),['fever']),   ## in a fever column with the help of SI we fill missing values by mean, mode, median.
                                              ('thf2', OrdinalEncoder(categories= [['Mild', 'Strong']]), ['cough']),          ## by this process we encode our data
                                              ('thf3', OneHotEncoder(sparse_output=False, drop= 'first'), ['gender', 'city'])],
                                              remainder = 'passthrough')   ## remainder = passthrough ==> it means reat all the columns remain same.

In [15]:
transformer.fit_transform(x_train).shape

(80, 7)

In [16]:
transformer.transform(x_test).shape

(20, 7)

## **TIPS Dataset**

In [17]:
import seaborn as sns

In [34]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [35]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns= ['total_bill']),
                                                            df['total_bill'], test_size = 0.2,
                                                            random_state = 8)

In [20]:
x_train

,tip,sex,smoker,day,time,size
1,1.66,Male,No,Sun,Dinner,3
33,2.45,Female,No,Sat,Dinner,4
189,4.00,Male,Yes,Sun,Dinner,3
132,1.50,Female,No,Thur,Lunch,2
140,3.50,Female,No,Thur,Lunch,2
...,...,...,...,...,...,...
133,2.00,Female,No,Thur,Lunch,2
105,1.64,Male,Yes,Sat,Dinner,2
241,2.00,Male,Yes,Sat,Dinner,2
84,2.03,Male,No,Thur,Lunch,2


In [33]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
x_train_day = lb.fit_transform(x_train['day']).reshape(-1,1)

x_train_test = lb.fit_transform(x_train['day'])

x_train_day.shape

(195, 1)

In [22]:
## Ordinal Encoding
oe = OrdinalEncoder(categories= [['Male', 'Female'], ['No', 'Yes'],['Dinner', 'Lunch']])
x_train_sex_smoker_time = oe.fit_transform(x_train[['sex', 'smoker', 'time']])

## for test column
x_test_sex_smoker_time = oe.fit_transform(x_test[['sex','smoker','time']])

x_train_sex_smoker_time.shape

(195, 3)

In [23]:
## Extraction
x_train_size = x_train.drop(columns=['sex', 'smoker', 'day', 'time']).values

x_test_size = x_train.drop(columns=['sex', 'smoker', 'day', 'time']).values

In [24]:
x_train_size.shape

(195, 2)

In [25]:
x_train_transformed = np.concatenate((x_train_day, x_train_sex_smoker_time, x_train_size), axis = 1)

In [26]:
x_train_transformed.shape

(195, 6)

## By using Column Transformer

In [42]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers=[('tnf1', OrdinalEncoder(categories=[['Sat', 'Sun', 'Thur', 'Fri']]), ['day']),
                                              ('tnf2', OrdinalEncoder(categories= [['Male', 'Female'], ['No', 'Yes'],['Dinner', 'Lunch']]),['sex', 'smoker', 'time'])
                                              ], remainder= 'passthrough')

In [43]:
transformer.fit_transform(x_train).shape

(195, 6)

## Titanic Data 

In [75]:
df = pd.read_csv(r"c:\Users\singh\OneDrive\Desktop\Python\Data\titanic - titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [76]:
df = df.drop(columns= ['Name', 'Cabin'])
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,892,0,3,male,34.5,0,0,330911,7.8292,Q
1,893,1,3,female,47.0,1,0,363272,7.0000,S
2,894,0,2,male,62.0,0,0,240276,9.6875,Q
3,895,0,3,male,27.0,0,0,315154,8.6625,S
4,896,1,3,female,22.0,1,1,3101298,12.2875,S


In [77]:
x = df.drop(columns= ['PassengerId', 'Ticket'])
y = df[['PassengerId', 'Ticket']]

In [78]:
df.isnull().sum()

PassengerId     0
Survived        0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Ticket          0
Fare            1
Embarked        0
dtype: int64

In [79]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=45)
x_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
388,0,3,male,21.00,0,0,7.7500,Q
72,1,3,female,29.00,0,0,7.9250,S
45,0,3,male,25.00,0,0,7.9250,S
173,0,3,male,NaN,0,0,7.2292,C
281,0,3,male,0.75,1,1,13.7750,S


In [80]:
x_train.dtypes

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

In [83]:
x_train['Sex'].value_counts()

Sex
male      219
female    115
Name: count, dtype: int64

In [70]:
x_train.shape

(334, 9)

In [64]:
## Applying Simple Imputer
si = SimpleImputer(strategy='mean')
x_train[['Age', 'Fare']] = si.fit_transform(x_train[['Age','Fare']])
x_train.shape

(334, 9)

In [84]:
x_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
388,0,3,male,21.00,0,0,7.7500,Q
72,1,3,female,29.00,0,0,7.9250,S
45,0,3,male,25.00,0,0,7.9250,S
173,0,3,male,NaN,0,0,7.2292,C
281,0,3,male,0.75,1,1,13.7750,S


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

tf = ColumnTransformer(
    transformers=[
        ('tnf1', OrdinalEncoder(categories=[['male', 'female'], ['Q', 'S', 'C']]),
         ['Sex', 'Embarked']),

         ('tnf2', SimpleImputer(strategy= 'mean'), 
          ['Age', 'Fare'])
    ],
    remainder='passthrough'
)

In [101]:
tf.fit_transform(x_train).shape

(334, 8)